**Motor imagery-based EEG signal processing**<br/>
Presented by: Reza Saadatyar  2022-2023 <br/>
E-mail: Reza.Saadatyar92@gmail.com  <br/>
[Link](https://github.com/RezaSaadatyar/Motor-imagery-based-EEG-signal-processing)


================================== Importing the required Libraries ===========================

In [2]:
import os
import numpy as np
import pandas as pd
from scipy import io
from itertools import chain
import matplotlib.pyplot as plt

========================================= Functions ==================================

In [3]:
# =============================================== Preparing data =========================================================
def Prepare_data(data, channels=None):  
    # ?------------------------------------------- Check type data -------------------------------------------      
    if 'DataFrame' not in str(type(data)):      
        data = pd.DataFrame(data)
    # !--------------------------------------- Check dimensional data ----------------------------------------
    if data.ndim < 3:
        if data.shape[0] < data.shape[1]:
            data = data.T
    elif data.ndim > 3:
        print("Please configure the data as two-dimensional (Sample * Channels)")
        sys.exit()
    # *--------------------------------------------- Set channels --------------------------------------------
    if channels is not None:
        data.columns=channels 
    else:
        data.columns = np.arange(1,data.shape[1]+1)
         
    return data, data.columns

*1. Importing the raw data*<br/>
[Dataset](https://www.bbci.de/competition/iv/desc_1.html)<br/>
*Parameters of data:*
  - *cnt:* the continuous EEG signals, size [time x channels]. The array is stored in datatype INT16. To convert it to uV values, use cnt= 0.1*double(cnt); in Matlab.
  - *mrk:* structure of target cue information with fields (the file of evaluation data does not contain this variable).
    - *pos:* vector of positions of the cue in the EEG signals given in unit sample, length #cues.
    - *y:* vector of target classes (-1 for class one or 1 for class two), length #cues.
  - *nfo:* structure providing additional information with fields
    - *fs:* sampling rate
    - *clab:* cell array of channel labels
    - *classes:* cell array of the names of the motor imagery classes
    - *xpos:* x-position of electrodes in a 2d-projection
    - *ypos:* y-position of electrodes in a 2d-projection

In [4]:
data_set = io.loadmat('BCICIV_calib_ds1a_100Hz.mat')
print(f"Variables = {list(data_set.keys())[3:]}")

Variables = ['cnt', 'mrk', 'nfo']


In [30]:
data = np.double(data_set['cnt']) * 0.1

Mrk = data_set['mrk']                   # Time of trials
Time_Trial = np.array(Mrk['pos'].tolist()).flatten()
Labels = np.array(Mrk['y'].tolist()).flatten()

Nfo = data_set['nfo']
Fs = Nfo['fs']
channels = list(chain(*list(np.array(Nfo['clab'].tolist()).flatten())))
Name_class = Nfo['classes']


(200,)

In [ ]:
B= Nfo['clab']
list(chain(*list(chain(*B))))

In [104]:
list(chain(*np.array(Nfo['clab'].tolist()).flatten()))

['AF3',
 'AF4',
 'F5',
 'F3',
 'F1',
 'Fz',
 'F2',
 'F4',
 'F6',
 'FC5',
 'FC3',
 'FC1',
 'FCz',
 'FC2',
 'FC4',
 'FC6',
 'CFC7',
 'CFC5',
 'CFC3',
 'CFC1',
 'CFC2',
 'CFC4',
 'CFC6',
 'CFC8',
 'T7',
 'C5',
 'C3',
 'C1',
 'Cz',
 'C2',
 'C4',
 'C6',
 'T8',
 'CCP7',
 'CCP5',
 'CCP3',
 'CCP1',
 'CCP2',
 'CCP4',
 'CCP6',
 'CCP8',
 'CP5',
 'CP3',
 'CP1',
 'CPz',
 'CP2',
 'CP4',
 'CP6',
 'P5',
 'P3',
 'P1',
 'Pz',
 'P2',
 'P4',
 'P6',
 'PO1',
 'PO2',
 'O1',
 'O2']

In [82]:

Name_channel = list(np.concatenate(list(np.array(Nfo['clab'].tolist()).flatten())))

X_position = list(np.array(Nfo['xpos'].tolist()).flatten())
Y_position = list(np.array(Nfo['ypos'].tolist()).flatten())

array([[array([[array(['AF3'], dtype='<U3'), array(['AF4'], dtype='<U3'),
                array(['F5'], dtype='<U2'), array(['F3'], dtype='<U2'),
                array(['F1'], dtype='<U2'), array(['Fz'], dtype='<U2'),
                array(['F2'], dtype='<U2'), array(['F4'], dtype='<U2'),
                array(['F6'], dtype='<U2'), array(['FC5'], dtype='<U3'),
                array(['FC3'], dtype='<U3'), array(['FC1'], dtype='<U3'),
                array(['FCz'], dtype='<U3'), array(['FC2'], dtype='<U3'),
                array(['FC4'], dtype='<U3'), array(['FC6'], dtype='<U3'),
                array(['CFC7'], dtype='<U4'), array(['CFC5'], dtype='<U4'),
                array(['CFC3'], dtype='<U4'), array(['CFC1'], dtype='<U4'),
                array(['CFC2'], dtype='<U4'), array(['CFC4'], dtype='<U4'),
                array(['CFC6'], dtype='<U4'), array(['CFC8'], dtype='<U4'),
                array(['T7'], dtype='<U2'), array(['C5'], dtype='<U2'),
                array(['C3'], dtype='<U

In [ ]:
Nfo = data_set['nfo']
Nfo

In [ ]:
data, channels = Prepare_data(data, channels=None)

In [ ]:



Fs = 100
Time = np.arange(Data.shape[0]) / Fs

